
Las inyecciones representan una de las vulnerabilidades más comunes y peligrosas en las aplicaciones web. Consisten en la inserción de código malicioso o no deseado en una aplicación vulnerable, con el objetivo de alterar su funcionamiento previsto. Este tipo de ataque puede ocurrir cuando una aplicación no valida o desinfecta correctamente los datos introducidos por los usuarios, lo que permite a un atacante ejecutar comandos maliciosos, obtener acceso no autorizado a datos, o incluso comprometer la seguridad del sistema.

## INYECCIONES LDAP

### Descripción

Una inyección LDAP (Lightweight Directory Access Protocol) es un tipo de ataque que aprovecha vulnerabilidades en las consultas LDAP de una aplicación para manipular o modificar los datos enviados a un servidor de directorio. Al igual que las inyecciones SQL, las inyecciones LDAP ocurren cuando las entradas de usuario no se validan ni desinfectan adecuadamente, permitiendo que un atacante inserte comandos o filtros LDAP maliciosos en las consultas.

El protocolo LDAP se utiliza comúnmente para acceder y gestionar información en directorios como Active Directory, que contienen datos sobre usuarios, dispositivos y otros recursos de red. Un atacante puede aprovechar una vulnerabilidad de inyección LDAP para:

Acceder a información no autorizada dentro del servidor de directorio.
Modificar datos en el directorio.
Bypass de autenticación para obtener acceso a recursos o cuentas restringidas.
Ejecutar operaciones no deseadas que comprometan la seguridad del sistema.

### Ejemplo de Inyección LDAP

`(&(uid=usuario)(password=clave))`

Si la aplicación no valida correctamente la entrada del usuario, un atacante podría introducir una cadena manipulada como:

`(&(uid=usuario)(&))(password=clave)`

En este caso, la consulta resultante siempre será verdadera, permitiendo que el atacante eluda la autenticación.

### Prevención
Para mitigar los riesgos de una inyección LDAP, es importante:

- Validar y desinfectar las entradas del usuario antes de utilizarlas en consultas LDAP.
- Usar consultas LDAP parametrizadas, lo que reduce la posibilidad de manipulación de los parámetros.
- Restringir los permisos de las aplicaciones en el servidor de directorios, limitando el acceso a solo la información necesaria.

In [3]:
#r "nuget: System.DirectoryServices"

using System;
using System.Text.RegularExpressions;
using System.DirectoryServices;

public class LdapHelper
{
    // Patrón de expresión regular para validar entradas LDAP
    private static readonly string ldapSafePattern = @"^[a-zA-Z0-9\-_]+$";

    // Método para validar la entrada
    public static bool IsValidLdapInput(string input)
    {
        return Regex.IsMatch(input, ldapSafePattern);
    }

    // Método para realizar la búsqueda LDAP
    public static void SearchLdap(string username, string password)
    {
        // Validación de la entrada
        if (!IsValidLdapInput(username) || !IsValidLdapInput(password))
        {
            throw new ArgumentException("Invalid input detected. Input contains unsafe characters.");
        }

        // Configuración de conexión LDAP (ejemplo)
        string ldapPath = "LDAP://your-ldap-server.com";
        using (DirectoryEntry directoryEntry = new DirectoryEntry(ldapPath))
        {
            // Autenticación con las credenciales proporcionadas
            directoryEntry.Username = username;
            directoryEntry.Password = password;

            try
            {
                // Intentar enlazar para validar las credenciales
                DirectorySearcher searcher = new DirectorySearcher(directoryEntry)
                {
                    Filter = $"(&(uid={username})(userPassword={password}))"
                };
                SearchResult result = searcher.FindOne();

                if (result != null)
                {
                    Console.WriteLine("User authenticated successfully.");
                }
                else
                {
                    Console.WriteLine("Invalid username or password.");
                }
            }
            catch (Exception ex)
            {
                Console.WriteLine($"LDAP error: {ex.Message}");
            }
        }
    }
}

Console.Write("Is alumno valid? ");
if (LdapHelper.IsValidLdapInput("alumno") == true)
    Console.WriteLine("Yes");
else
    Console.WriteLine("No");

Console.Write("Is 123abc. valid? ");
if (LdapHelper.IsValidLdapInput("123abc.") == true)
    Console.WriteLine("Yes");
else
    Console.WriteLine("No");

Console.Write("Is user123)(|(uid=*)) valid? ");
if (LdapHelper.IsValidLdapInput("user123)(|(uid=*))") == true)
    Console.WriteLine("Yes");
else
    Console.WriteLine("No");


Installed Packages System.DirectoryServices, 8.0.0

Is alumno valid? Yes
Is 123abc. valid? No
Is user123)(|(uid=*)) valid? No


### MAS INFORMACION

- https://www.blackhat.com/presentations/bh-europe-08/Alonso-Parada/Whitepaper/bh-eu-08-alonso-parada-WP.pdf
- https://www.exploit-db.com/docs/spanish/21315-[spanish]-taller-de-inyecciones-ldap.pdf
